In [1]:
import random
import math
import json
import os
import pandas as pd

In [2]:
jobs = pd.read_csv('data\\csv\\jobs.csv')
skills = pd.read_csv('data\\csv\\skills.csv')
monsters = pd.read_csv('data\\csv\\monsters.csv')

In [219]:
# 캐릭터 공통 클래스
class Character(object):
    def __init__(self, name, level, max_hp, hp, strength, dex):
        self.name = name
        self.level = level
        self.max_hp = max_hp
        self.hp = hp
        self.strength = strength
        self.dex = dex

    def attack(self):
        return self.strength * 3 + self.strength * random.randint(0, 2)

    def take_damage(self, damage):
        self.hp -= damage

    def is_alive(self):
        return self.hp > 0

# 유저(플레이어) 클래스
class Player(Character):
    def __init__(self, name):
        level = 1
        max_hp = 150
        hp = max_hp
        strength = 5
        dex = 5
        int = 5
        super().__init__(name, level, max_hp, hp, strength, dex)
        self.exp = 0
        self.dex = 5
        self.int = 5

    def gain_exp(self, exp):
        self.exp += exp

    def required_exp(self):
        return 10 + round(self.level ** 1.5)

    def level_up(self):
        self.strength += 1
        self.dex += 1
        self.int += 1
        self.max_hp += 20
        self.exp -= self.required_exp()
        self.level += 1

    def to_dict(self):
        return {
            'name': self.name,
            'level': self.level,
            'max_hp': self.max_hp,
            'strength': self.strength,
            'dex' : self.dex,
            'int' : self.int,
            'exp' : self.exp,
        }

    def save(self, filename):
        # 기존 데이터가 있는 경우 로드
        if os.path.exists(filename):
            with open(filename, 'r') as f:
                data = json.load(f)
        else:
            data = {}

        # 플레이어 정보 저장
        data[self.name] = self.to_dict()

        with open(filename, 'w') as f:
            json.dump(data, f, indent=4, ensure_ascii=False)

    @classmethod
    def load(cls, name, filename='players.json'):
        if not os.path.exists(filename):
            raise FileNotFoundError('저장된 플레이어 파일이 없습니다.')

        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if name not in data:
            raise ValueError(f"'{name}' 플레이어 파일이 없습니다.")

        info = data[name]
        player = cls(name)
        player.level = info['level']
        player.max_hp = info['max_hp']
        player.strength = info['strength']
        player.dex = info['dex']
        player.int = info['int']
        player.exp = info['exp']
        return player

# 몬스터 클래스
class Enemy(Character):
    def __init__(self, name, level):
        strength = level + 2
        dex = level + 2
        max_hp = 140 +level * 18
        hp = max_hp
        super().__init__(name, level, max_hp, hp, strength, dex)
        self.exp_reward = 3 + round(level ** 1.2)

In [220]:
# 로그인 또는 캐릭터 생성
username = input('캐릭터 아이디를 입력하세요. 새로운 아이디가 입력될 경우 신규 캐릭터가 생성됩니다.')
try:
    player = Player.load(username, 'players.json')
    print(f'{username}님의 정보를 불러왔습니다.')
except(FileNotFoundError, ValueError) as e:
    print(f'{username}님은 처음 접속하셨습니다. 신규 캐릭터를 생성합니다.')
    player = Player(username)
    player.save('players.json')

hg님의 정보를 불러왔습니다.


In [221]:
# 전투
def battle(player, enemy):
    log = []
    turn = 1
    player.hp = player.max_hp
    while player.is_alive() and enemy.is_alive():
        log.append(f"{turn}턴입니다.\n")

        # 플레이어 선공
        evasion_chance = max(0.05, min((enemy.dex - player.dex) * 0.03, 0.8))
        if random.random() < evasion_chance:
            log.append("적이 회피했습니다!\n")
        else:
            player_dmg = player.attack()
            enemy.take_damage(player_dmg)
            log.append(f"{player_dmg}의 피해를 입혔습니다!\n")

        # 몬스터 처치시 보상 획득
        if not enemy.is_alive():
            log.append("전투에서 승리했습니다!\n")

            player.gain_exp(enemy.exp_reward)
            log.append(f"{enemy.exp_reward}의 경험치를 획득했습니다.")

            if player.exp >= player.required_exp():
                player.level_up()
                log.append("\n레벨 업! 올스탯 1과 최대hp 10이 증가합니다.")

            player.save('players.json')
            break

        # 몬스터 후공
        evasion_chance = max(0.05, min((player.dex - enemy.dex) * 0.03, 0.8))
        if random.random() < evasion_chance:
            log.append('적의 공격을 회피했습니다!\n')
        else:
            enemy_dmg = enemy.attack()
            player.take_damage(enemy_dmg)
            log.append(f"{enemy_dmg}의 피해를 입었습니다!\n")
        if not player.is_alive():
            log.append("전투에서 패배했습니다.")
            player.hp = player.max_hp
            player.save('players.json')
            break
        turn += 1
    return log


In [224]:
# 몬스터 설정
enemy = Enemy("goblin", 25)

# 전투 테스트
test_battle = battle(player, enemy)
print(''.join(test_battle))

1턴입니다.
144의 피해를 입혔습니다!
적의 공격을 회피했습니다!
2턴입니다.
108의 피해를 입혔습니다!
108의 피해를 입었습니다!
3턴입니다.
144의 피해를 입혔습니다!
81의 피해를 입었습니다!
4턴입니다.
144의 피해를 입혔습니다!
135의 피해를 입었습니다!
5턴입니다.
144의 피해를 입혔습니다!
전투에서 승리했습니다!
51의 경험치를 획득했습니다.


In [204]:
# 캐릭터 정보 확인 함수
def view_user_stat(player):
    print(f"캐릭터명 : {player.name}")
    print(f"레벨 : {player.level}")
    print(f"최대 hp : {player.max_hp}")
    print(f"힘 : {player.strength}")
    print(f"민첩 : {player.dex}")
    print(f"지능 : {player.int}")


In [206]:
view_user_stat(player)

캐릭터명 : hg
레벨 : 29
최대 hp : 380
힘 : 33
민첩 : 33
지능 : 33


In [ ]:
'''
최대mp, 현재mp 스탯추가
스킬 딕셔너리 만들기 : 각 스탯 계수 설정
레벨업시 올릴 스탯 선택
공격력, 방어력, 마공, 마방, 회피율, 크리율 등 파생스탯 생성
인벤토리/아이템 추가(장비, 소비)
전투시 행동 선택(공격/스킬 사용/아이템 사용 등)
등장 몬스터 종류 다양화(랜덤 또는 제한적으로 선택가능하도록), 종류별 스탯도 다르게
전투종료시 hp/mp 회복량 조정
사망시 레벨 및 찍은 스탯 초기화/마법과 아이템은 계승
층 올라가는 탑 형태로 연속전투 설계
10층마다 보스(클리어시 특별보상). 몬스터 및 보스별 드랍테이블 필요
보스는 스킬 사용 등 특별 패턴
새 게임 시작시 들어갈 층 선택 가능, 미입력시 1층부터
버프/디버프/상태이상
전투 외 확률적인 이벤트 발생(부활찬스, 특별한 스킬/아이템 획득 등)
층마다 전투가 좀더 길고 전략적 선택이 가능하도록 설계
마법/아이템/전투방식에 시너지가 발생하도록 설계
확률적 요소가 강하게 작동하도록 설계
골드 획득(계승하지 않도록 해서 전략적인 활용/죽었을 때 상실감을 주는 요소로 쓰고 싶음)
아이템 상점
장비 강화(후순위)
보스 난이도는 좀 어렵게, 기믹을 파훼하기 위해 특정 아이템이나 스킬 등 준비가 필요하도록 설계
게임 시작시 로드할 캐릭터, 몬스터 정보 등은 csv파일로 관리, 중첩구조가 필요한 데이터 등은 json으로 관리

'''
